# Versuch 1: Statistik I: Würfel-Simulation

In [3]:
#Name: Finn Jannik Wagner
#Matrikelnummer: 8102237
#E-Mail-Adresse: finn.jannik.wagner@physik.uni-giessen.de
#Studiengang: Physik
#Wiederholen Sie das Praktikum (Ja/Nein)? Nein

Die folgenden Aufgaben basieren auf der Generierung von (Pseudo-)Zufallszahlen am Computer. Daher liefert ein mehrfaches Ausführen der Versuche unterschiedliche Werte. Führen Sie den Code jeder einzelnen Aufgabe daher mindestens dreimal aus und dokumentieren Sie die Ergebnisse. Nutzen Sie dann (wie auch bei Experimenten im Labor) den Mittelwert und die Standardabweichung, wenn Sie die Fragen in Ihrem Protokoll beantworten.

##### Aufgabenstellung: 
Für die folgenden Aufgaben werden die Ergebnisse von Würfelwürfen untersucht. <br><br>
a) Berechnen Sie zunächst (auf Papier) den Erwartungswert, sowie die Standardabweichung eines 6-seitigen Würfels, für die Augensumme von i) zwei iii) fünf und iv) zehn Würfeln.  <br><br>
b) Berechnen Sie zunächst den Erwartungswert, sowie die Standardabweichung eines 20-seitigen Würfels, für i) die Augenzahl eines Würfels, sowie für die Augensumme von ii) zwei iii) fünf und iv) zehn Würfeln.  <br><br>
c) Simulieren Sie mit Hilfe der Funktion dice(n, m = 1, k = 20, save = "example") ein Zufallsexperiment. Dabei soll ein 20-seitiger Würfel i) $n = 100$ mal, ii) $n = 1000$ und iii) $n = 10000$ mal geworfen werden. Vergleichen Sie jeweils den Mittelwert und die empirische Standardabweichung mit dem Erwartungswert und der Standardabweichung aus Teil a). Anmerkung: "example" ist ein Platzhalter für den Namen der PDF-Datei, welche abgespeichert wird. Verwenden Sie unterschiedliche Namen, sonst wird die Datei überschrieben.<br><br>
d) Simulieren Sie mit Hilfe der Funktion dice(n=1000, m, k=20, save = "example") ein Zufallsexperiment. Dabei sollen i) zwei, ii) fünf und iii) zehn 20-seitige Würfel je 1000 mal geworfen werden. Vergleichen Sie jeweils den Mittelwert und die empirische Standardabweichung mit dem Erwartungswert und der Standardabweichung aus Teil a). <br><br>
e) Interpretieren Sie Ihre Ergebnisse. Entsprechen Sie Ihren Erwartungen?  <br><br>
f) Führen Sie mit den Werten aus d) mit Hilfe der Funktion chi_search(x = points, sigma, mu, save = "example") einen Fit durch. Dabei wird eine Gaußfunktion als theoretische Funktion verwendet. Vergleichen sie die $\chi ^2$ Werte. Welcher Wert ist besonders hoch/gering und woran könnte das liegen? <br><br>
g) Wie hoch ist die Wahrscheinlichkeit, dass beim 10-maligen Werfen eines 20-seitigen Würfels i) mindestens einmal die 20 fällt? ii) höchstens 2 mal die 1 fällt? iii) genau 3 mal die 5 fällt? iv) genau 3 mal die 5 UND höchstens 2 mal die 1 (Tipp: Baumdiagramm kann helfen)


##### Bemerkung: 
Sie können gerne auch mit anderen Werten herumspielen, um ein besseres Gefühl für die Zusammenhänge zu bekommen. Verwenden Sie in diesem Fall die entsprechende Funktion ohne das save-Argument, also dice(n, m, k) anstatt dice(n, m, k, save). Für die Auswertung sind der Werte aus der Aufgabenstellung relevant.

In [ ]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings

In [ ]:
def dice (n, m, k, save = False):                                       
    
    """
    dice(n, m, k, save) 
    n = Anzahl der Würfelwürfe (Integer), 
    m = Anzahl der Würfel (Integer), 
    k = Anzahl der Würfelseiten (Integer) 
    save = Name des PDFs (String)
    
    Funktion plottet Histogramm eines simulierten Würfelexperiments, inklusive Markierung für den Mittelwert
    und gibt den Array, den Mittelwert und die Standardabweichung der 
    Augensummen aus. Zumdem wird ein PDF erstellt, falls save übergeben wird
    """
    m1 = m
    k1 = k
    dice_matrix=np.array([])                                            #leeres Array zum Auffüllen
    for i in range (n):
        dice_row=np.array([])
        for j in range (m):
            dice_row=np.append(dice_row,[rnd.randint(1,k+1)])           #m-er Reihe wird erzeugt. Gefüllt durch Randoms
        dice_matrix= np.append(dice_matrix,[dice_row])                  #n-te m-er Reihe wird dem Array hinzugefügt
    dice_sum = sum(dice_matrix.reshape(n,m).transpose())                #Summe der Augenzahlen
    plt.hist(dice_sum, bins = (k-1)*m+1, 
             range = (m, m*k), color = 'limegreen' , density = True)    #Histogram mit relativen Häufigkeit
    plt.axvline(np.mean(dice_sum),color='black' ,linestyle='--')        #vertikale Linie des Mittelwertes
    plt.xlabel("Augenzahl")                                             #Bezeichnung x-Achse
    plt.ylabel("Relative Häufigkeit")                                   #Bezeichnung y-Achse
    if (save != False):                                                 #wenn save ein String s ist, wird ein PDF mit dem Namen s gespeichert
        plt.title(save)
        plt.savefig(save+'.pdf')
    plt.show()                                                          #Plot
    print ("Arithmetisches Mittel: ", np.mean(dice_sum), "\nEmpirische Standardabweichung: ", np.std(dice_sum) )              
    return (dice_sum, np.mean(dice_sum), np.std(dice_sum), k1, m1 )             #Array, Mittelwert und Standardabweichung der Augensummen


def gauss(sigma, mu, x):
    a = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-0.5* (x - mu)**2 /sigma**2)
    return a   

def chi_squared(y, sigma, mu, save = False):
    x = y[0]
    k = y[3]
    m = y[4]
    A = plt.hist(x, bins = np.arange(m,m*k+2), density = True)
    a= sum((((gauss(sigma, mu, A[1][:-1]) - A[0]) **2) /gauss(sigma, mu, A[1][:-1])))
    plt.plot(np.linspace(0, m*k+2, 1000), gauss(sigma, mu, np.linspace(0, m*k+2, 1000)), 'r')
    plt.xlabel("Augenzahl")
    plt.ylabel("Relative Häufigkeit")
    if (save != False):
        plt.title(save)
        plt.savefig(save+'.pdf')
    plt.show()
    print("chi-Quadrat:", a )
    pass


In [ ]:
#Aufgabe 1a)

"""
Erwartungswerte von 6-seitigen Würfeln
1 Würfel:
    Alle 6 Werte des Würfels sind gleich wahrscheinlich mit der Wahrscheinlichkeit 1/6.
    Der Erwartungswert µ ist damit 1*1/6 + 2 * 1/6 + 3*1/6 + 4*1/6 + 5*1/6 + 6*1/6 = µ = 3.5
    
    Die Standardabweichung ist
    math.sqrt(1/6*(3.5 - 1)**2 + 1/6*(3.5 - 2)**2 + 1/6*(3.5 - 3)**2 + 1/6*(3.5 - 4)**2 + 1/6*(3.5 - 5)**2 + 1/6*(3.5 - 6)**2) = math.sqrt(17.5/6) ≈ 1.71
    
2 Würfel:
    Da einzelne Würfelwürfe unabhängig von einander sind und für den Erwartungswert Linearität gilt, ist der Erwartungswert für n Würfel n*3.5
    In diesem Fall also µ = 2*3.5 = 7
    
    Die Standardabweichung ist
    math.sqrt( 1/36*(7 - 2)**2 + 2/36*(7 - 3)**2 + 3/36*(7 - 4)**2 + 4/36*(7 - 5)**2 + 5/36*(7 - 6)**2 +
    6/36*(7 - 7)**2 + 5/36*(7 - 8)**2 + 4/36*(7 - 9)**2 + 3/36*(7 - 10)**2 + 2/36*(7 - 11)**2 + 1/36*(7 - 12)**2) = math.sqrt(35/6) = math.sqrt(2 * 17.5/6) ≈ 2.42
    Für die Varianz gilt bei unabhängigen Würfelwürfen ebenfalls Linearität, diese lässt sich da die Wurzelfunktion monoton ist auch auf die Standardabweichung übertragen.
    Die Standardabweichung für zwei Würfel ist die eines Würfels mal den Faktor math.sqrt(2).
    
5 Würfel:
    Erwartungswert µ = 5*3.5 = 17.5
    Standardabweichung σ = math.sqrt(5 * 17.5/6) ≈ 3.82
    
10 Würfel:
    Erwartungswert µ = 10*3.5 = 25
    Standardabweichung σ = math.sqrt(10 * 17.5/6) ≈ 5.4

"""

In [ ]:
#Aufgabe 1b)

"""
Wir nehmen an das sich bei dem Würfel um einen Ikosaeder handelt, alle Seiten also gleich wahrscheinlich mit 1/20 sind.

1 Würfel:
    Der Erwartungswert ist damit µ = 1*1/20 + 2*1/20 + 3*1/20 + ... = 1/20 * (1+2+3+...) = (20+1)/2 * 20 * 1/20 = 21/2 = 10.5
    Die Standardabweichung ist σ = math.sqrt(1/20 * sum([(10.5 - i)**2 for i in range(1, 20+1)])) = math.sqrt(133/4) ≈ 5.77

Äquivalent zu einem 6 seitigen Würfel lassen sich Erwartungswert und Standardabweichung für mehrere Würfe auf Grund von Monotonie einfach bestimmen.

2 Würfel:
    Erwartungswert µ = 2*10.5 = 21
    Standardabweichung σ = math.sqrt(2 * 133/4) ≈ 8.15
    
5 Würfel:
    Erwarungswert µ = 5*10.5 = 52.5
    Standardabweichung σ = math.sqrt(5 * 133/4) ≈ 12.89

10 Würfel:
    Erwarungswert µ = 10*10.5 = 105
    Standardabweichung σ = math.sqrt(10 * 133/4) ≈ 18.23
"""

In [ ]:
#Aufgabe 1c)

"""
Simulieren Sie mit Hilfe der Funktion dice(n, m = 1, k = 20, save = "example") ein Zufallsexperiment.
Dabei soll ein 20-seitiger Würfel i) 𝑛=100 mal, ii) 𝑛=1000 und iii) 𝑛=10000 mal geworfen werden.
Vergleichen Sie jeweils den Mittelwert und die empirische Standardabweichung mit dem Erwartungswert und der Standardabweichung aus Teil a).
Anmerkung: "example" ist ein Platzhalter für den Namen der PDF-Datei, welche abgespeichert wird. Verwenden Sie unterschiedliche Namen, sonst wird die Datei überschrieben.
"""

dice1 = dice(n=100, m=1, k=20) # Ergebnis für 100 Würfe
dice2 = dice(n=1000, m=1, k=20) # Ergebnis für 1000 Würfe
dice3 = dice(n=10000, m=1, k=20) # Ergebnis für 10000 Würfe
print(dice1, dice2, dice3)

"""
Ergebenise der Zufallsexperimente TODO MINDESTENS DREIMAL AUSFÜHREN
100 Würfe:
    Arithmetisches Mittel:  10.54 
    Empirische Standardabweichung:  5.740069685988141
1000 Würfe:
    Arithmetisches Mittel:  10.606 
    Empirische Standardabweichung:  5.896504388194756
10000 Würfe:
    Arithmetisches Mittel:  10.5459 
    Empirische Standardabweichung:  5.7362263893608665
"""


In [ ]:
# Wie berechnet man den Mittelwert dreier Ergebnisse?
# Anleitung: schreiben Sie z.B. (3.1+3.2+3.3)/3
#Hier ein Testbeispiel
(3.1+3.2+3.3)/3

In [ ]:
# Berechnen Sie den Mittelwert für Ihre Ergebnisse aus 1 c)
#Schreiben Sie hier Ihren eigenen Code angelehnt an das vorherige Beispiel

# Der Mittelwert der drei Experimente mit verschiedenen Anzahlen an Würfen ist:
(10.54 + 10.606 + 10.5459)/3  # ≈ 10.563967
# Oder als geschlossener Ausdruck
(dice(n=100, m=1, k=20)[1] + dice(n=1000, m=1, k=20)[1] + dice(n=10000, m=1, k=20)[1])/3


? #Auswertung Mittelwert für 100 Würfe
"""
Der Mittelwert (Arithmetisches Mittel) ist mit 10.54 eine gute Aproximation des berechneten Werts von 10.5.
Der eperimentele Wert ist auf eine Nachkommastelle genau mit einer Abweichung von 0.04 (abs(10.5-10.54))

Experimentell lassen sich auch drei Simulationen mit 100 Würfeln machen und ihre Mittelwerte mitteln:
"""
µ = (dice(n=100, m=1, k=20)[1] + dice(n=100, m=1, k=20)[1] + dice(n=100, m=1, k=20)[1])/3
# Eingesetzt; Mittelwert
# (10.39 + 10.87 + 10.4)/3 = 10.553¯
# Abstand
abs(10.5-µ)
# abs(10.5-10.553¯) = 0.053

In [ ]:
? #Auswertung Mittelwert für 1000 Würfe
"""
Der Mittelwert (Arithmetisches Mittel) ist mit 10.606 eine gute Aproximation des berechneten Werts von 10.5.
Der eperimentele Wert hat eine Abweichung vom berechneten von 0.106 (abs(10.5-10.606)). Diese Abweichung ist größer als die des Experimentes mit weniger Würfen,
ein gutes Beispiel das in einigen Fällen auch unwahrscheinlichere Verteilungen auftreten können.

Experimentell lassen sich auch drei Simulationen mit 1000 Würfeln machen und ihre Mittelwerte mitteln:
"""
µ = (dice(n=1000, m=1, k=20)[1] + dice(n=1000, m=1, k=20)[1] + dice(n=1000, m=1, k=20)[1])/3
# Eingesetzt
(10.346 + 10.163 + 10.539)/3 = 10.3493¯
# Abstand
abs(10.5-µ)
# abs(10.5-10.3493¯) = 0.1507

In [ ]:
? #Auswertung Mittelwert für 10000 Würfe
"""
Der Mittelwert (Arithmetisches Mittel) ist mit 10.5459 eine gute Aproximation des berechneten Werts von 10.5.
Der eperimentele Wert hat eine Abweichung vom berechneten von 0.0459 (abs(10.5-10.5459)). Diese Abweichung nähert sich im Vergleich zum Experiment mit 1000 Würfen wieder
dem exakten Mittelwert, hat aber immer noch eine höhere Abweichung als das Experiment mit 100 Würfen. Wahrlich ein paradehaftes Beispiel für Abweichungen bei Wahrscheinlichkeiten

Experimentell lassen sich auch drei Simulationen mit 1000 Würfeln machen und ihre Mittelwerte mitteln:
"""
µ =(dice(n=10000, m=1, k=20)[1] + dice(n=10000, m=1, k=20)[1] + dice(n=10000, m=1, k=20)[1])/3
# Eingesetzt
(10.454 + 10.5954 + 10.5301)/3 = 10.52649¯
# Abstand
abs(10.5-µ)
# abs(10.5-10.52649¯) = 0.02649

In [ ]:
# Berechnen Sie die Standardabweichung für Ihre Ergebnisse aus (a)
# Anleitung: wenn 3.2 Ihr Mittelwert war, dann schreiben Sie
# np.sqrt((3.0-3.2)**2+(3.1-3.2)**2+(3.3-3.2)**2)
#Hier ein Testbeispiel
np.sqrt((3.1-3.2)**2+(3.2-3.2)**2+(3.4-3.2)**2)

In [ ]:
# Berechnen Sie die Standardabweichung für Ihre Ergebnisse aus 1 c)
#Schreiben Sie hier Ihren eigenen Code angelehnt an das vorherige Beispiel

? #Auswertung Standardabweichung für 3 Versuche mit 100, 1000 und 10000 Würfen
# np.sqrt((v_1-µ)**2+(v_2-µ)**2+(v_3-µ)**2)
# Standardabweichung mit n=100, n=1000 und n=10000 nach den vorherigen Zahlen
# Mit µ = 10.563967 als Mittel der Mittelwerte
# und v_1 = 10.54, v_2 = 10.606, v_3 = 10.5459
σ = np.sqrt((10.54-10.563967)**2+(10.606-10.563967)**2+(10.5459-10.563967)**2) # ≈ 0.05146
# Sigma ist hier die Standardabweichung der 3 experimentellen Werte untereinander und zum theoretischen Wert

# Oder allgemein
v_1 = dice(n=100, m=1, k=20)[1]
v_2 = dice(n=1000, m=1, k=20)[1]
v_3 = dice(n=10000, m=1, k=20)[1]
µ = (v_1 + v_2 + v_3)/3
σ = np.sqrt((v_1-µ)**2+(v_2-µ)**2+(v_3-µ)**2)

? #Auswertung Standardabweichung für 100 Würfe
v_1 = dice(n=100, m=1, k=20)[1]
v_2 = dice(n=100, m=1, k=20)[1]
v_3 = dice(n=100, m=1, k=20)[1]
µ = (v_1 + v_2 + v_3)/3
σ = np.sqrt((v_1-µ)**2+(v_2-µ)**2+(v_3-µ)**2)
# Eingesetzt für Werte 10.39, 10.87, 10.4
σ = np.sqrt((10.39-10.563967)**2+(10.87-10.563967)**2+(10.4-10.563967)**2) # ≈ 0.388

In [ ]:
? #Auswertung Standardabweichung für 1000 Würfe
v_1 = dice(n=1000, m=1, k=20)[1]
v_2 = dice(n=1000, m=1, k=20)[1]
v_3 = dice(n=1000, m=1, k=20)[1]
µ = (v_1 + v_2 + v_3)/3
σ = np.sqrt((v_1-µ)**2+(v_2-µ)**2+(v_3-µ)**2)
# Eingesetzt für Werte 10.346, 10.163, 10.539
σ = np.sqrt((10.346-10.563967)**2+(10.163-10.563967)**2+(10.539-10.563967)**2) # ≈ 0.457

In [ ]:
? #Auswertung Standardabweichung für 1000 Würfe
v_1 = dice(n=10000, m=1, k=20)[1]
v_2 = dice(n=10000, m=1, k=20)[1]
v_3 = dice(n=10000, m=1, k=20)[1]
µ = (v_1 + v_2 + v_3)/3
σ = np.sqrt((v_1-µ)**2+(v_2-µ)**2+(v_3-µ)**2)
# Eingesetzt für Werte 10.454, 10.5954, 10.5301
σ = np.sqrt((10.454-10.563967)**2+(10.5954-10.563967)**2+(10.5301-10.563967)**2) # ≈ 0.119

In [ ]:
#Aufgabe 1d)
"""
Simulieren Sie mit Hilfe der Funktion dice(n=1000, m, k=20, save = "example") ein Zufallsexperiment. 
Dabei sollen i) zwei, ii) fünf und iii) zehn 20-seitige Würfel je 1000 mal geworfen werden.
Vergleichen Sie jeweils den Mittelwert und die empirische Standardabweichung mit dem Erwartungswert und der Standardabweichung aus Teil a). 
"""
dice4 = dice(n=1000, m=2, k=20) #Ergebnis für 2 Würfel
dice5 = dice(n=1000, m=5, k=20) #Ergebnis für 5 Würfel
dice6 = dice(n=1000, m=10, k=20) #Ergebnis für 10 Würfel
print(dice4, dice5, dice6)

"""
Ergebenise der Zufallsexperimente:
 
2 Würfel:
    Arithmetisches Mittel:  21.228 
    Empirische Standardabweichung:  7.998500859536117

5 Würfel:
    Arithmetisches Mittel:  52.391 
    Empirische Standardabweichung:  12.966268507168898

10 Würfel:
    Arithmetisches Mittel:  104.764 
    Empirische Standardabweichung:  18.34285430351558
"""

In [ ]:
? #Auswertung Mittelwert für 2 Würfel
"""
Theoretisches Ergebniss
2 Würfel:
    Erwartungswert µ = 2*10.5 = 21
    Standardabweichung σ = math.sqrt(2 * 133/4) ≈ 8.15

Der experimentelle Erwartungswert hat eine Abweichung von 0.228 (abs(21-21.228)) vom exakten Wert
"""
# Experimenteller Erwartungswert und Abweichung vom exakten Erwartungswert
µ = dice(n=1000, m=2, k=20)[1]
abs(21 - µ)

In [ ]:
? #Auswertung Mittelwert für 5 Würfel
"""
5 Würfel:
    Erwarungswert µ = 5*10.5 = 52.5
    Standardabweichung σ = math.sqrt(5 * 133/4) ≈ 12.89
    
Der experimentelle Erwartungswert hat eine Abweichung von 0.109 (abs(52.5-52.391)) vom exakten Wert
"""
# Experimenteller Erwartungswert und Abweichung vom exakten Erwartungswert
µ = dice(n=1000, m=5, k=20)[1]
abs(52.5 - µ)

In [ ]:
? #Auswertung Mittelwert für 10 Würfel
"""
10 Würfel:
    Erwarungswert µ = 10*10.5 = 105
    Standardabweichung σ = math.sqrt(10 * 133/4) ≈ 18.23
    
Der experimentelle Erwartungswert hat eine Abweichung von 0.236 (abs(105-104.764)) vom exakten Wert
"""
# Experimenteller Erwartungswert und Abweichung vom exakten Erwartungswert
µ = dice(n=1000, m=10, k=20)[1]
abs(105 - µ)

In [ ]:
? #Auswertung Standardabweichung für 2 Würfel
"""
Theoretisches Ergebniss
2 Würfel:
    Erwartungswert µ = 2*10.5 = 21
    Standardabweichung σ = math.sqrt(2 * 133/4) ≈ 8.15

Die experimentelle Standardabweichung hat eine Abweichung von 0.15625235561392792 (abs(math.sqrt(2 * 133/4) - 7.998500859536117)) vom exakten Wert
"""
# Experimentelle Standardabweichung und Abweichung von der exakten Standardabweichung
σ = dice(n=1000, m=2, k=20)[2]
abs(math.sqrt(2 * 133/4) - σ)

In [ ]:
? #Auswertung Standardabweichung für 5 Würfel
"""
Theoretisches Ergebniss
5 Würfel:
    Erwarungswert µ = 5*10.5 = 52.5
    Standardabweichung σ = math.sqrt(5 * 133/4) ≈ 12.89

Die experimentelle Standardabweichung hat eine Abweichung von 0.07247154894127128 (abs(math.sqrt(5 * 133/4) - 12.966268507168898)) vom exakten Wert
"""
# Experimentelle Standardabweichung und Abweichung von der exakten Standardabweichung
σ = dice(n=1000, m=5, k=20)[2]
abs(math.sqrt(5 * 133/4) - σ)

In [ ]:
? #Auswertung Standardabweichung für 10 Würfel
"""
Theoretisches Ergebniss
10 Würfel:
    Erwarungswert µ = 10*10.5 = 105
    Standardabweichung σ = math.sqrt(10 * 133/4) ≈ 18.23

Die experimentelle Standardabweichung hat eine Abweichung von 0.10827177470510918 (abs(math.sqrt(10 * 133/4) - 18.34285430351558)) vom exakten Wert
"""
# Experimentelle Standardabweichung und Abweichung von der exakten Standardabweichung
σ = dice(n=1000, m=10, k=20)[2]
abs(math.sqrt(10 * 133/4) - σ)

In [ ]:
#Aufgabe 1e) 

"""
Interpretieren Sie Ihre Ergebnisse. Entsprechen Sie Ihren Erwartungen? 

Ja, die Werte entsprachen meinen Erwartungen. Die theoretisch berechneten Werte unterscheiden sich bei den hier durch gerecheneten Beispielversuchen erst
in der ersten Nachkommastelle. Folglich sind wohl Theorie und experimentelle Umsetzung korrekt durchgeführt worden, da sich die Ergebnise, bis auf ebenjene Abweichungen, gleichen.

Die Ergebnisse der durchgeführten Versuche wurden wiedererwartens nicht konstant mit steigender Durchführungszahl immer genauer.
Die Abweichungen beim wiederholten Werfen des 20-seitigen Würfels waren 0.04, 0.106 und 0.0459. Hier ist sogar das erste Ergebnis bei dem nur 100 mal geworfen wurde
näher am theoretischen Wert als beide Versuche mit 10 und 100 mal so vielen Würfen.
Ebenso gibt es bei den berechneten Standardabweichungen einen Ausreißer da der Wert für 3 Versuche mit 1000 größer ist, als für 100
Die Standardabweichungen bei den wiederholten Versuchen waren auch relativ klein.
Um also aus Experimenten Gesetzmäßigkeiten herzuleiten ist es also unbedingt von Vorteil möglichst viele Experimente durchzuführen um einen möglichst genauen Mittelwert zu bestimmen.
TODO Mehr
"""

In [ ]:
#Aufgabe 1f)
"""
Führen Sie mit den Werten aus d) mit Hilfe der Funktion chi_search(x = points, sigma, mu, save = "example") einen Fit durch.
Dabei wird eine Gaußfunktion als theoretische Funktion verwendet. Vergleichen sie die 𝜒2 Werte.
Welcher Wert ist besonders hoch/gering und woran könnte das liegen? 

Ergebenise der Zufallsexperimente:
 
2 Würfel:
    Arithmetisches Mittel:  21.228 
    Empirische Standardabweichung:  7.998500859536117

5 Würfel:
    Arithmetisches Mittel:  52.391 
    Empirische Standardabweichung:  12.966268507168898

10 Würfel:
    Arithmetisches Mittel:  104.764 
    Empirische Standardabweichung:  18.34285430351558
"""

chi_squared(dice4, sigma = 7.998500859536117, mu=21.228)
chi_squared(dice5, sigma = 12.966268507168898, mu=52.391)
chi_squared(dice6, sigma = 18.34285430351558, mu=104.764)
TODO AUFGABE BEARBEITEN

In [ ]:
#Aufgabe 1g)

"""
Wie hoch ist die Wahrscheinlichkeit, dass beim 10-maligen Werfen eines 20-seitigen Würfels 
i) mindestens einmal die 20 fällt? 
ii) höchstens 2 mal die 1 fällt? 
iii) genau 3 mal die 5 fällt? 
iv) genau 3 mal die 5 UND höchstens 2 mal die 1
(Tipp: Baumdiagramm kann helfen)

i) Wir betrachten die Wahrscheinlichkeit das die 20 überhaupt nicht fällt.
In jedem Wurf bleiben von den 20 Möglichkeiten noch 19 übrig. q ist damit = (1 - p)**10 = (19/20)**10
Die gesuchte Wahrscheinlichkeit ist dann p = 1 - q = 1 - (19/20)**10 ≈ 0.401 

ii) 
Die Wahrscheinlichkeit auf eine 1 ist p = 1/20
Wir betrachten nun die Wahrscheinlichkeiten das die Eins 0 mal, 1 mal oder 2 mal gewürfelt wird.
Die Wahrscheinlichkeit das bei n Würfen die Zahl, mit Wahrscheinlichkeit p, genau k-mal vorkommt ist:
p(x=k) = n!/(k! * (n-k)!) * p**k * (1-p)**(n-k)

Die Wahrscheinlichkeit das die gar nicht 1 fällt ist (19/20)**10 ≈ 0.599
p(k=0) = math.factorial(10)/(math.factorial(0) * math.factorial(10-0)) * (1/20)**0 * (1-1/20)**(10-0) ≈ 0.599
p(k=1) = math.factorial(10)/(math.factorial(1) * math.factorial(10-1)) * (1/20)**1 * (1-1/20)**(10-1) ≈ 0.315
p(k=2) = math.factorial(10)/(math.factorial(2) * math.factorial(10-2)) * (1/20)**2 * (1-1/20)**(10-2) ≈ 0.075

Die gesuchte Wahrscheinlichkeit ist damit P = p(k=0) + p(k=1) + p(k=2) ≈ 0.988496

iii)
Die Wahrscheinlichkeit das genau 3 mal die 5 fällt lässt sich mit der Formel aus Aufgabenteil ii) berechnen
p(k=3) = math.factorial(10)/(math.factorial(3) * math.factorial(10-3)) * (1/20)**3 * (1-1/20)**(10-3) ≈ 0.0105

iv)
Die beiden Ereignisse ii) und iii) sind stochastisch unabhängig dann p(ii) * p(iii) = 
    math.factorial(10)/(math.factorial(3) * math.factorial(10-3)) * (1/20)**3 * (1-1/20)**(10-3) * (
        math.factorial(10)/(math.factorial(0) * math.factorial(10-0)) * (1/20)**0 * (1-1/20)**(10-0) +
        math.factorial(10)/(math.factorial(1) * math.factorial(10-1)) * (1/20)**1 * (1-1/20)**(10-1) +
        math.factorial(10)/(math.factorial(2) * math.factorial(10-2)) * (1/20)**2 * (1-1/20)**(10-2)
    )
≈ 0.01038

Die beiden Ereignisse sind stochastisch abhängig:
Damit das Ereignis eintritt müssen 3 Würfel eine 5 zeigen. Die übrigen 7 Würfel können nun 0, 1 oder 2 Einsen zeigen.
Also p(iii) * (p(k=0) + p(k=1) + p(k=2))
math.factorial(10)/(math.factorial(3) * math.factorial(10-3)) * (1/20)**3 * (1-1/20)**(10-3) * (
    math.factorial(7)/(math.factorial(0) * math.factorial(7-0)) * (1/20)**0 * (1-1/20)**(7-0) +
    math.factorial(7)/(math.factorial(1) * math.factorial(7-1)) * (1/20)**1 * (1-1/20)**(7-1) +
    math.factorial(7)/(math.factorial(2) * math.factorial(7-2)) * (1/20)**2 * (1-1/20)**(7-2)
) ≈ 0.01043570419298467


TODO Richtig???
"""